# Creating a Map displaying all the flights I have ever taken and more....

### Bringing in the required packages

In [51]:
reset -sf

In [ ]:
whos

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
##movie stuff 
# Uncomment to save
ffmpeg_path = cwd + '\\ffmpeg-20170204-b1e2192-win64-static\bin'
plt.rcParams['animation.ffmpeg_path'] = ffmpeg_path + '\\ffmpeg'
mywriter = animation.FFMpegWriter(fps = 70)

#mywriter = animation.FFMpegWriter(fps = 70)
#anim.save('mymovie.mp4',writer=mywriter)


In [20]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, writer=mywriter, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

In [21]:
from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [ ]:
2+3

In [ ]:
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import os
import matplotlib.animation as animation
from matplotlib.lines import Line2D

### Sort the filenames out

In [5]:
cwd = os.getcwd()
print(cwd)
in_filename = cwd + '\Data\SW-J flight travel.csv'
print(in_filename)

C:\users\vz28\Mapping project\python mapping
C:\users\vz28\Mapping project\python mapping\Data\SW-J flight travel.csv


### Load the data in

In [6]:
csv_cols = ('flight', 'dep_coutry', 'dep_airport','dep_airport_code','dep_lat','dep_lon',
                'arr_country','arr_airport','arr_airport_code','arr_lat','arr_lon',
                'date', 'notes','trip_name')

routes = pd.read_csv(in_filename,names= csv_cols, skiprows=1)
routes.head(2)

,flight,dep_coutry,dep_airport,dep_airport_code,dep_lat,dep_lon,arr_country,arr_airport,arr_airport_code,arr_lat,arr_lon,date,notes,trip_name
0,1,UK,Heathrow,LHR,51.477500,-0.461389,United States,Los Angeles,LAX,33.942536,-118.408075,01/08/2000,estimated date,Family United States 1
1,2,United States,San Francisco,SFO,37.618972,-122.374889,UK,Heathrow,LHR,51.477500,-0.461389,15/08/2000,estimated date,Family United States 1


### Colour set up

In [7]:
bg_color = (0.0, 0.0, 0, 1.0)
coast_color = (204/255.0, 0, 193/255.0, .9)
color = (1.0, 0.80000000000000004, 0.90196078431372551, 1.0)

In [8]:
#set initial conditions
m = Basemap(projection='mill', lon_0=0, resolution= 'l') 
m.drawcoastlines(color=coast_color, linewidth=1.5)
m.fillcontinents(color=bg_color, lake_color=bg_color)
m.drawmapboundary(fill_color=bg_color)
lines = []

### Populate the lines list

In [7]:
route

NameError: name 'route' is not defined

In [11]:
m.drawgreatcircle(route['dep_lon'], route['dep_lat'],
                                  route['arr_lon'], route['arr_lat'],linewidth=2.0, color= color)


NameError: name 'route' is not defined

In [9]:
for i, route in enumerate(routes.sort_values(by='flight',
                              ascending=True).iterrows()):
    route = route[1]
    line, = m.drawgreatcircle(route['dep_lon'], route['dep_lat'],
                                  route['arr_lon'], route['arr_lat'],linewidth=2.0, color= color)
    lines.append(line)  
    

D:\Anaconda3\lib\site-packages\mpl_toolkits\basemap\__init__.py:3274: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
D:\Anaconda3\lib\site-packages\mpl_toolkits\basemap\__init__.py:3283: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)


### "lines" now contains our flight paths

### Set up the animation figure

In [10]:
fig2 = plt.figure(figsize=(27, 20))
m2 = Basemap(projection='mill', lon_0=0, resolution= 'l')
m2.drawcoastlines(color=coast_color, linewidth=1.5)
m2.fillcontinents(color=bg_color, lake_color=bg_color)
m2.drawmapboundary(fill_color=bg_color)      

### Initialise the flight path

In [11]:
current_line = 0
current_point = 0
x,y = lines[current_line].get_data()
x= x[0]
y= y[0]
flight_path = [[],[],[]]
for i in iter(range(len(lines))):
        line = m2.plot(x,y, color=color)[0]
        line_head = m2.plot(x,y, color='red', marker='o', markeredgecolor='r')[0]
        line_head_slice = m2.plot(x,y, color='red', linewidth=2)[0]
        line.set_data(x, y)
        flight_path[0].append(line)
        flight_path[1].append(line_head)           #head
        flight_path[2].append(line_head_slice)     #head slice
        

D:\Anaconda3\lib\site-packages\mpl_toolkits\basemap\__init__.py:3274: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
D:\Anaconda3\lib\site-packages\mpl_toolkits\basemap\__init__.py:3283: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)


### Define Init to be used if repeat set to true

In [12]:
def init():
    #print "data is", repr(iter(range(len(lines))))
    global  current_line , current_point, flight_path
    current_line = 0
    current_point = 0
    x,y = lines[current_line].get_data()
    x= x[0]
    y= y[0]
    for i in iter(range(len(lines))):
        flight_path[0][i].set_data(x,y)
        flight_path[1][i].set_data(x,y)       #head
        flight_path[2][i].set_data(x,y)    #head slice
    return flight_path[0] + flight_path[1] + flight_path[2]

### Calclulate how many frames do we need

In [13]:
s=0
for i in range(len(lines)):
    s += len(lines[i].get_data()[0])
print ('frame count = ' + str(s))

frame count = 2463


### Define the animation function

In [14]:
wx = [0,1,2,3,4,5,6,7,8,9,10,11,12]
wx[:13]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [15]:
r = 0
def animate(frames):
#def animate(frames, *args, **kwargs):
    global  current_line , current_point, flight_path, lines,r
    head_len = 10
    x,y = lines[current_line].get_data()
    
    flight_path[0][current_line].set_data(x[:current_point+1],y[:current_point+1])
    flight_path[1][current_line].set_data(x[current_point],y[current_point])
    if (current_point < 10):
        
        flight_path[2][current_line].set_data(x[:current_point+1],y[:current_point+1])
        
    if (current_point > 9):
        flight_path[2][current_line].set_data(x[(current_point-head_len):current_point],
                                              y[(current_point-head_len):current_point])    
        
    
    if (current_point == lines[current_line].get_data()[0].size -1):
        
        current_point = 0
        flight_path[2][current_line].set_data([],[])
        current_line += 1
        
    else:
        current_point += 1   
    r += 1
    return flight_path[0] + flight_path[1] + flight_path[2]

### Call the animator.  blit=True means only re-draw the parts that have changed.

In [16]:
anim = animation.FuncAnimation(fig2, animate, frames=s-1, interval=5, blit=True, init_func=init, repeat= False)
print(r)

0


In [67]:
##movie stuff 
# Uncomment to save
#ffmpeg_path = cwd + '\\ffmpeg-20160924-267da70-win64-static\\bin'
plt.rcParams['animation.ffmpeg_path'] = ffmpeg_path + '\\ffmpeg'
#mywriter = animation.FFMpegWriter(fps = 70)
#anim.save('mymovie.mp4',writer=mywriter)
plt.show()

# that's it for now folks

In [26]:
#plt.rcParams
# animation.ImageMagickWriter.isAvailable()

In [17]:
display_animation(anim)

D:\Anaconda3\lib\site-packages\matplotlib\animation.py:971: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)


ValueError: Cannot save animation: no writers are available. Please install ffmpeg to save animations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.ffmpeg_path'] = 'D:/Programme/ffmpeg/bin/ffmpeg.exe'
plt.rcParams['savefig.bbox'] = 'tight'     # tight garbles the video!!!

movie_file = 'D:/Daten/basic_animation.mp4'
mywriter = animation.FFMpegWriter()

fig = plt.figure()
fig.set_size_inches(4, 5, True)
ax = plt.axes(xlim=(0, 2), ylim=(-2, 2))
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               interval = 1000, frames = 20, blit=True)

anim.save(movie_file, dpi=100, fps = 30, writer=mywriter, 
          extra_args=['-vcodec', 'libx264'])

plt.show()

RuntimeError: Passing in values for arguments fps, codec, bitrate, extra_args, or metadata is not supported when writer is an existing MovieWriter instance. These should instead be passed as arguments when creating the MovieWriter instance.